# Simple Logistic Regression Model

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="whitegrid", context='notebook', color_codes=True)

from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression

In [4]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data"
names = ["preg", "plas", "pres", "skin", "test", "mass", "pedi", "age", "class"]
df = pd.read_csv(url, names=names)

In [5]:
df.head(5)

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.describe()

,preg,plas,pres,skin,test,mass,pedi,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
df.shape

(768, 9)

df.values

In [21]:
X = df.values[:, 0:8]

In [22]:
y = df.values[:, 8]

In [24]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.20)

In [28]:
model = LogisticRegression()
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print(score)
filename = "diabetes_model.sav"
pickle.dump(model, open(filename, 'wb'))

0.766233766234


In [27]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.766233766234


In [29]:
loaded_model.predict(X[0, 0:8])

/Users/joseph/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 1.])

In [30]:
X[0, 0:8]

array([   6.   ,  148.   ,   72.   ,   35.   ,    0.   ,   33.6  ,
          0.627,   50.   ])

In [33]:
loaded_model.predict(np.array([6., 148., 72.,35.,0.,33.6,0.627,50.]))

/Users/joseph/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 1.])

In [34]:
loaded_model.predict([6., 148., 72.,35.,0.,33.6,0.627,50.])

/Users/joseph/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 1.])

# Flask Api of Model